In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

 #Configure pandas for high dimensionality dataframes
pd.set_option('display.max_columns', None)

In [2]:
#Imports
cust_train = pd.read_csv(r'C:\Users\Public\cust_train.csv')
cust_test = pd.read_csv(r'C:\Users\Public\cust_test.csv')
print(cust_train.info(), cust_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2627 non-n

In [3]:
#Preview data
cust_train.head(3)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B


In [4]:
#Remove ID column
cust_train = cust_train.drop(cust_train[['ID']], axis=1)

In [5]:
#Check train label balance proportions
cust_train['Segmentation'].value_counts(normalize=True)

Segmentation
D    0.281111
A    0.244422
C    0.244175
B    0.230293
Name: proportion, dtype: float64

In [6]:
#Count missing values
cust_train.isna().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [7]:
#Drop records with missing values
cust_train = cust_train.dropna()
print("Records after dropping missing values: "+str(len(cust_train)))

Records after dropping missing values: 6665


In [8]:
cust_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6665 entries, 0 to 8067
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           6665 non-null   object 
 1   Ever_Married     6665 non-null   object 
 2   Age              6665 non-null   int64  
 3   Graduated        6665 non-null   object 
 4   Profession       6665 non-null   object 
 5   Work_Experience  6665 non-null   float64
 6   Spending_Score   6665 non-null   object 
 7   Family_Size      6665 non-null   float64
 8   Var_1            6665 non-null   object 
 9   Segmentation     6665 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 572.8+ KB


In [9]:
#Convert float cols to int
cust_train.astype({'Work_Experience':int, 'Family_Size':int})

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Male,No,22,No,Healthcare,1,Low,4,Cat_4,D
2,Female,Yes,67,Yes,Engineer,1,Low,1,Cat_6,B
3,Male,Yes,67,Yes,Lawyer,0,High,2,Cat_6,B
5,Male,Yes,56,No,Artist,0,Average,2,Cat_6,C
6,Male,No,32,Yes,Healthcare,1,Low,3,Cat_6,C
...,...,...,...,...,...,...,...,...,...,...
8062,Male,Yes,41,Yes,Artist,0,High,5,Cat_6,B
8064,Male,No,35,No,Executive,3,Low,4,Cat_4,D
8065,Female,No,33,Yes,Healthcare,1,Low,1,Cat_6,D
8066,Female,No,27,Yes,Healthcare,1,Low,4,Cat_6,B


In [10]:
#Updated balance
cust_train['Segmentation'].value_counts(normalize=True)

Segmentation
D    0.263616
C    0.258065
A    0.242461
B    0.235859
Name: proportion, dtype: float64

In [11]:
#Detect categorical object/string columns
obj_tr = (cust_train.dtypes == 'object')
obj_cols_tr = list(obj_tr[obj_tr].index)
obj_cols_tr

['Gender',
 'Ever_Married',
 'Graduated',
 'Profession',
 'Spending_Score',
 'Var_1',
 'Segmentation']

In [12]:
#List categories and levels
cat_count=0
for col in obj_cols_tr:
    print(pd.DataFrame(cust_train[col].value_counts()))
    cat_count += len(cust_train[col].value_counts())
print('\nTotal # categorical levels/values: ', cat_count)

        count
Gender       
Male     3677
Female   2988
              count
Ever_Married       
Yes            3944
No             2721
           count
Graduated       
Yes         4249
No          2416
               count
Profession          
Artist          2192
Healthcare      1077
Entertainment    809
Doctor           592
Engineer         582
Executive        505
Lawyer           500
Marketing        233
Homemaker        175
                count
Spending_Score       
Low              3999
Average          1662
High             1004
       count
Var_1       
Cat_6   4476
Cat_4    849
Cat_3    634
Cat_2    362
Cat_7    166
Cat_1    104
Cat_5     74
              count
Segmentation       
D              1757
C              1720
A              1616
B              1572

Total # categorical levels/values:  29


In [13]:
#Encode categorical variables depending on relationships between values

 #LabelEncoder for label ('Segmentation'); maps labels alphabetically (A=0, D=3)
enc_lab = LabelEncoder()
 #OrdinalEncoder for 'Spending_Score
  #Assign Spend_Score categories to continuous values; Low=1, high=3
enc_ord = OrdinalEncoder(categories=[['skip','Low','Average','High']])
 #OneHotEncoder for remaining categorical features
enc_OneH = OneHotEncoder(sparse_output=False, dtype=int)

In [14]:
#Encode labels
cust_train['Segmentation'] = enc_lab.fit_transform(cust_train['Segmentation'])
cust_train['Segmentation'].head(3)

0    3
2    1
3    1
Name: Segmentation, dtype: int32

In [15]:
#Encode 'Spending_Score'
cust_train['Spending_Score'] = enc_ord.fit_transform(cust_train[['Spending_Score']])
cust_train['Spending_Score'].head(3)

0    1.0
2    1.0
3    3.0
Name: Spending_Score, dtype: float64

In [16]:
#Confirm transformations
cust_train.head(3)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Male,No,22,No,Healthcare,1.0,1.0,4.0,Cat_4,3
2,Female,Yes,67,Yes,Engineer,1.0,1.0,1.0,Cat_6,1
3,Male,Yes,67,Yes,Lawyer,0.0,3.0,2.0,Cat_6,1


In [17]:
#Remaining categorical vars (not label or ordinal)
cols_rem = [col for col in obj_cols_tr if col not in ['Spending_Score', 'Segmentation']]
cols_rem

['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Var_1']

Before continuing and exploding the remaining string attributes out into separate one-hot columns, lets look at a scatter matrix of the numerical features

In [ ]:
pd.plotting.scatter_matrix(cust_train, grid=True, figsize=(10,10))
plt.show()

In [ ]:
#Encode features in dataframe
encoded = pd.DataFrame(enc_OneH.fit_transform(cust_train[['Gender','Ever_Married','Graduated','Profession','Var_1']]), columns=enc_OneH.get_feature_names_out())
 #Preview One-Hot features
print(encoded.shape)
encoded.head()

In [ ]:
#Review cust_train index after dropping records
print(cust_train.shape)
cust_train.head(3)

In [ ]:
#Reset cust_train index before concatenating to prevent record corruption due to incongruent indexes
cust_train.reset_index(drop=True, inplace=True)
print(cust_train.shape)
cust_train.head(3)

In [ ]:
#Combine original and one hot encoded dataframes
cust_train = pd.concat([cust_train, encoded], axis=1)
 #Drop pre-transformation features
cust_train = cust_train.drop(cust_train[['Gender','Ever_Married','Graduated','Profession','Var_1']], axis=1)

In [ ]:
#Resetting index before concatenation prevented 1146 rows from corrupting as they did previously
cust_train.isna().sum()

In [ ]:
#Move Label to last column
cust_train['Segmentation'] = cust_train.pop('Segmentation')

In [ ]:
#Preview full pre-processed df
cust_train.head(3)

In [ ]:
#Review summary statistics
cust_train.describe()

In [ ]:
#Create correlation matrix
corr_matrix = cust_train.corr()

#Plot masked heatmap
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
plt.figure(figsize=(18,12))

sns.heatmap(corr_matrix, annot=True, cmap='seismic', mask=mask, vmin=-1, vmax=1)
plt.title('Correlation Coefficients')
plt.show()

We can observe some simple associations in the correlation coefficients:
- Married people, lawyers, people who have graduated, and (to some degree) people in the anonymized Var_1 category '6' are correlated with greater ages
  - Healthcare workers tend to be younger
- Married people, lawyers, and exectives correlate with higher spending scores
  - Unmarried people and healthcare workers tend to spend less
 
Now lets scale the data and make some cluster predictions

In [ ]:
#Save column names
col_names = pd.Series(cust_train.columns)

#Scale data + reapply column titles
scaler = StandardScaler()

cust_train_sc = pd.DataFrame(scaler.fit_transform(cust_train))
cust_train_sc.rename(columns=col_names, inplace=True)

cust_train_sc.head(3)

In [ ]:
#Split labelled train data into train (6/7) and test (1/7) sets
split_index = len(cust_train)//7

y_tr = cust_train.iloc[:-split_index, -1:]
y_tr = y_tr.to_numpy().reshape(len(y_tr),)

X_tr = cust_train.iloc[:-split_index, :-1]

y_te = cust_train.iloc[-split_index:, -1:]
y_te = y_te.to_numpy().reshape(len(y_te),)

X_te = cust_train.iloc[-split_index:, :-1]

In [ ]:
X_tr.shape, y_tr.shape

In [ ]:
X_te.shape, y_te.shape

In [ ]:
#Get baseline accuracy for simple logistic regression model
base_log_reg = LogisticRegression(max_iter=2_500)
base_log_reg.fit(X_tr, y_tr)

In [ ]:
base_log_reg.score(X_te, y_te)

We're going to project our features onto a lower dimensional subspace in order to reduce the dimensionality of our dataset from 26.\
Let's determine how many principal components are required to retain 95% of the features' variance

In [ ]:
pca = PCA()
pca.fit(X_tr)

In [ ]:
#Output variance explained by each component
expl_var = pca.explained_variance_ratio_
ex_var_df = pd.DataFrame(expl_var)
ex_var_df

The orders of magnitude change from tenths to hundredths and beyond pretty fast, suggesting a few components account for much of the variance;\
lets see those values normalized in an array.

In [ ]:
#Output cumulative variance explained by principal components (first, second, etc.)
cumulative_sum = np.cumsum(expl_var)
cumulative_sum

We can see that at least 95% of the variance is captured by two principal components;\
99% is captured by three

In [ ]:
#Store components necessary to capture 95% of variance in variable
dim_95 = np.argmax(cumulative_sum >= 0.95) + 1
dim_99 = np.argmax(cumulative_sum >= 0.99) + 1

print(str(dim_95)+'\n'+str(dim_99))

Now, lets use a simpler method

In [ ]:
pca_95 = PCA(n_components=0.95)
pca_99 = PCA(n_components=0.99)

And transform our training dataset into two and then three principal components with PCA

In [ ]:
#Create reduced/PCA-transformed datasets
X_tr_2D = pca_95.fit_transform(X_tr)
X_tr_3D = pca_99.fit_transform(X_tr)

print('Number of components for 95% variance:', pca_95.n_components_)
print('Number of components for 99% variance:', pca_99.n_components_)

In [ ]:
#Plot 2D dataset
print(X_tr_2D.shape)
X_tr_2D

In [ ]:
plt.scatter(X_tr_2D[:, 0], X_tr_2D[:, 1], c='teal', alpha=0.5)

In [ ]:
#Plot 3D dataset
X_tr_3D.shape

In [ ]:
x = X_tr_3D[:, 0]
y = X_tr_3D[:, 1]
z = X_tr_3D[:, 2]

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(x, y, z, c="blue", marker="o" )
ax.set_title("Projection of principal components onto 3D subspace")
plt.show()

Above, we can visually confirm that this data might not need a third dimension